In [1]:
from python.new_data import *
import altair as alt
from python.chart_helpers import *

extractor = DataExtractor()

# lineouts_df = extractor.extract_lineouts_data()
# player_apps_df = extractor.extract_player_appearances()
# games_df = extractor.extract_games_data()
# pitchero_df = extractor.extract_pitchero_stats()
set_piece_df = extractor.extract_set_piece_stats()

In [2]:
set_piece_df

,game_id,team,set_piece,won,lost,total
0,2021-09-04_1st_Horsham_II,EG,lineout,15,1,16
1,2021-09-04_1st_Horsham_II,Opp,lineout,11,2,13
2,2021-09-04_1st_Horsham_II,EG,scrum,8,0,8
3,2021-09-04_1st_Horsham_II,Opp,scrum,3,1,4
4,2021-09-25_1st_Eastbourne,EG,lineout,15,2,17
...,...,...,...,...,...,...
375,2025-02-22_2nd_Burgess_Hill,Opp,scrum,4,0,4
376,2025-03-22_2nd_Horsham,EG,lineout,9,1,10
377,2025-03-22_2nd_Horsham,Opp,lineout,7,3,10
378,2025-03-22_2nd_Horsham,EG,scrum,7,0,7


In [1]:
from python.database import DatabaseManager
from python.new_data import *
import altair as alt
from python.chart_helpers import *

db = DatabaseManager()

db.load_source_data()

Created table: games
Created table: player_appearances
Created table: lineouts
Created table: pitchero_stats
Created table: set_piece_stats
Loaded 156 games


INFO:python.new_data:Fetching Pitchero stats for 2016/17 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=42025


Loaded 3083 player appearances


INFO:python.new_data:Fetching Pitchero stats for 2017/18 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=47693
INFO:python.new_data:Fetching Pitchero stats for 2018/19 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=52304
INFO:python.new_data:Fetching Pitchero stats for 2019/20 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=68499
INFO:python.new_data:Fetching Pitchero stats for 2021/22 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=79578
INFO:python.new_data:Fetching Pitchero stats for 2022/23 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=83980
INFO:python.new_data:Fetching Pitchero stats for 2023/24 1nd XV
INFO:python.new_data:URL: https://www.egrfc.com/teams/142068/statistics?season=87941
INFO:python.new_data:Fetching Pitchero stats for 2024/25 1nd XV
INFO:python.new_data:URL: https://www.egrf

Loaded 5610 Pitchero stats
Loaded 376 set piece stats
Source data loaded successfully


# Appearances Chart

In [9]:
other_names = {
    "S Cooke": "Steve Cooke",
    "R Andrews": "Ruari Andrews",
    "R Perry": "Ross Perry",
    "J Stokes": "James Stokes",
    "C Champain": "Callum Champain",
    "M Dewing": "Max Dewing",
    "A Schofield": "Ali Schofield",
    "J Gibbs": "Johnny Gibbs",
    "M Taylor": "Mark Taylor",
    "M Evans": "Max Evans",
    "J Carr": "Josh Carr",
    "Z Roberts": "Zach Roberts",
    "W Blackledge": "Will Blackledge",
    "T Sandys": "Tom Sandys",
    "B Swadling": "Ben Swadling",
    "O Waite": "Oscar Waite",
    "S Anderson": "Scott Anderson",
    "M Ansboro": "Martyn Ansboro",
    "M Tomkinson": "Matt Tomkinson",
    "B Meyerratken": "Ben Meyerratken",
    "L Cammish": "Leo Cammish",
    "C Lear": "Charlie Lear",
}

def clean_name(name):

    name_dict = {
        "Sam Lindsay": "S Lindsay 2",
        "Sam Lindsay-McCall": "S Lindsay",
        "James Mitchell": "T Mitchell",
    }

    if name in name_dict:
        return name_dict[name]

    initial = name.split(" ")[0][0]
    surname = " ".join(name.split(" ")[1:])
    surname = surname.replace("’", "'")
    name_clean = f"{initial} {surname}"
    # trim and title case
    return name_clean.strip().title()

df = db.con.execute(
"""
SELECT 
    P.player, 
    G.squad,
    G.season,
    G.game_type,
    P.position,
    P.unit,
    P.is_starter,
    IF(P.is_starter, 'Start', IF(NOT(P.is_starter), 'Bench', 'Unknown')) AS start,
    COUNT(*) AS games
FROM player_appearances P
LEFT JOIN games G USING (game_id)
GROUP BY P.player, G.squad, G.season, G.game_type, P.position, P.unit, P.is_starter
ORDER BY games DESC
""").df()

df_pitchero_historic = db.con.execute(
"""
SELECT 
    player_join,
    squad,
    season,
    MAX(A) AS games,
    NULL AS game_type,
    NULL AS position,
    NULL AS unit,
    'Unknown' AS start
FROM pitchero_stats
WHERE season < '2020/21'
GROUP BY player_join, squad, season
""").df()

df['player_join'] = df['player'].apply(clean_name)

name_lookup = df.set_index('player_join')['player'].to_dict()
name_lookup.update(other_names)

df.drop(columns=['player_join'], inplace=True)

# map pitchero names to player names
df_pitchero_historic['player_join'] = df_pitchero_historic['player_join'].map(lambda x: name_lookup[x] if x in name_lookup else x)
df_pitchero_historic.rename(columns={'player_join': 'player'}, inplace=True)

df = pd.concat([df, df_pitchero_historic], ignore_index=True)

# Add text mark showing total games per player, calculated dynamically in the chart spec
text = alt.Chart(df).mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=12
).encode(
    y=alt.Y('player:N', sort="-x"),
    x=alt.X('sum(games):Q'),
    text=alt.Text('sum(games):Q')
)

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("sum(games):Q", axis=alt.Axis(title=None, orient="top")),
    y=alt.Y("player:N", sort="-x", title=None),
    color=alt.Color('squad:N', title='Squad', scale=squad_scale, legend=alt.Legend(orient="right")),
    opacity=alt.Opacity(
        'start:N',
        scale=alt.Scale(
            domain=['Start', 'Bench', 'Unknown'],
            range=[1.0, 0.7, 0.4]
        ),
        legend=None
    ),
    order=alt.Order('squad:Q', sort='ascending'),
    tooltip=[
        alt.Tooltip('player:N', title='Player'),
        alt.Tooltip('squad:N', title='Squad'),
        alt.Tooltip('sum(games):Q', title='Games'),
        alt.Tooltip('start:N', title='Started?'),
    ]
).properties(
    width=400,
    height=alt.Step(15),
    title=alt.Title(text='Player Appearances', subtitle='Based on available data. Positions not shown for historic data (pre-2020/21).')
) + text

chart.save('python/player_appearances.json')


# Captains Chart

In [3]:
import altair as alt
from python.chart_helpers import *

df = db.con.execute(
"""
SELECT 
    P.player, 
    G.squad,
    G.season,
    G.game_type,
    P.is_captain,
    COUNT(*) AS games
FROM player_appearances P
LEFT JOIN games G USING (game_id)
WHERE P.is_captain OR P.is_vc
GROUP BY P.player, G.squad, G.season, G.game_type, P.is_captain 
ORDER BY games DESC
""").df()

alt.Chart(df).mark_bar().encode(
    x=alt.X("sum(games):Q", axis=alt.Axis(title=None, orient="top")),
    y=alt.Y("player:N", sort="-x", title=None),
    color=alt.Color("is_captain:N",
        scale=alt.Scale(domain=[True, False], range=["#202946", "#7d96e8"]),
        legend=alt.Legend(title=None, orient="none", legendX=300, legendY=50, labelExpr="datum.value ? 'Captain' : 'VC'"),
    ),
    order=alt.Order('is_captain:N', sort='descending'),
    row=alt.Row('squad:N', title=None, header=alt.Header(title=None, labelFontSize=36, labelExpr="datum.value + ' XV'"), spacing=50),
    tooltip=['player', 'squad', 'sum(games)', 'is_captain']
).resolve_scale(
    y='independent'
).properties(
    title=alt.Title("Match Day Captains", subtitle="Captains and Vice-Captains (if named)."),
    width=400,
    height=alt.Step(15)
).save('python/captains.json')

# Points Scorers Chart

In [49]:
df = db.con.execute(
"""
SELECT 
    A.player,
    P.*
FROM pitchero_stats P 
LEFT JOIN (SELECT DISTINCT player, player_join FROM player_appearances) A
ON A.player_join = P.player_join
WHERE A.player IS NOT NULL
AND count > 0
AND event IN ('T', 'Con', 'PK', 'DG')
""").df()

df["Points"] = df.apply(lambda x: x["count"] * (5 if x["event"] == "T" else 2 if x["event"] == "Con" else 3 if x["event"] in ["PK", "DG"] else 0), axis=1)
df["event"] = df["event"].map(lambda x: "Try" if x == "T" else "Pen" if x == "PK" else x)

score_selection = alt.selection_point(fields=['event'], bind='legend')

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("sum(Points):Q", axis=alt.Axis(title="Points", orient="top")),
    y=alt.Y("player:N", sort="-x", title=None),
    color=alt.Color(
        'event:N', 
        scale=alt.Scale(
            domain=["Try", "Con", "Pen"],
            range=["#202946", "#7d96e8", "#d62728"]
        ),
        legend=alt.Legend(title='Click to filter', orient="none", legendX=300, legendY=100)
    ),
    order=alt.Order('event_sort:Q', sort='ascending'),
    tooltip=[
        alt.Tooltip('player:N', title='Player'),
        alt.Tooltip('event:N', title='Type'),
        alt.Tooltip('sum(count):Q', title='Count'),
        alt.Tooltip('sum(Points):Q', title='Points'),
        alt.Tooltip('sum(A):Q', title='Games'),
    ]   
).add_params(score_selection).transform_filter(score_selection).transform_calculate(
    event_sort = "datum.event == 'Try' ? 1 : datum.event == 'Con' ? 2 : datum.event == 'Pen' ? 3 : 4"
).properties(
    width=400,
    height=alt.Step(15),
    title=alt.Title(text='Point Scorers', subtitle='Based on available Pitchero data. Click on legend to filter by score type.')
)

chart.save('python/point_scorers.json')

# Cards Chart

In [5]:
df = db.con.execute(
"""
SELECT 
    A.player,
    P.*
FROM pitchero_stats P 
LEFT JOIN (SELECT DISTINCT player, player_join FROM player_appearances) A
ON A.player_join = P.player_join
WHERE A.player IS NOT NULL
AND count > 0
AND event IN ('RC', 'YC')
""").df()

df["event"] = df["event"].map(lambda x: "Red" if x == "RC" else "Yellow" if x == "YC" else x)

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("sum(count):Q", axis=alt.Axis(title=None, orient="top", format='d')),
    y=alt.Y("player:N", sort="-x", title=None),
    color=alt.Color(
        'event:N', 
        scale=alt.Scale(
            domain=["Red", "Yellow"],
            range=["#d62728", "goldenrod"]
        ),
        legend=alt.Legend(title='Click to filter', orient="none", legendX=300, legendY=100)
    ),
    tooltip=[
        alt.Tooltip('player:N', title='Player'),
        alt.Tooltip('event:N', title='Card Type'),
        alt.Tooltip('sum(count):Q', title='Cards'),
        alt.Tooltip('sum(A):Q', title='Games'),
    ]   
).properties(
    width=400,
    height=alt.Step(15),
    title=alt.Title(text='Cards', subtitle='Based on available Pitchero data. Click on legend to filter by card type.')
)

chart.save('python/cards.json')

# Team Sheets

In [6]:
import altair as alt



# Use db to query player_apps_df joined with games_df in SQL, return as pandas DataFrame
df = db.con.execute(
"""
SELECT DISTINCT
    P.*,
    G.game_id,
    G.date,
    G.opposition,
    G.season,
    G.game_type,
    G.squad,
    G.home_away,
    REPLACE(CONCAT_WS(' ', G.game_id, CONCAT('(', G.home_away, ')')), '_', ' ') AS game_label,
    CONCAT_WS(' ', G.opposition, CONCAT('(', G.home_away, ')')) AS game_label_short,
    CONCAT_WS(' ', P.position, CONCAT('(', P.shirt_number, ')')) AS position_label,
    CONCAT_WS(
        ' ',
        SPLIT_PART(P.player, ' ', 1), -- FIRST NAME
        ARRAY_TO_STRING(
            LIST_TRANSFORM(SPLIT(SPLIT_PART(P.player, ' ', 2), '-'), x -> SUBSTR(x, 1, 1)),
            ''
        )
    ) AS player_label
FROM player_appearances P
LEFT JOIN games G USING (game_id)
ORDER BY G.date DESC, P.shirt_number ASC
""").df()

player_position_agg = df.groupby(['player', 'position']).size().reset_index(name='count')
player_primary_position = player_position_agg.loc[player_position_agg.groupby('player')['count'].idxmax()][['player', 'position']]

# Merge primary position back into main df
df = df.merge(player_primary_position, on=['player'], how='left', suffixes=('', '_primary'))

# Selection for player highlighting
player_highlight = alt.selection_point(
    fields=['player'], 
    on="mouseover",
    clear='mouseout',
    empty='all'
)
click_player_highlight = alt.selection_point(
    fields=['player'], 
    on="click",
    clear='dblclick',
    empty='all'
)

# Format date as string (e.g., '2024-08-23' -> '23 Aug 2024')
df['date_str'] = df['date'].dt.strftime('%d %b %Y')

# Use .encode() with explicit types and field names, avoid ambiguous axis titles
chart = alt.Chart(df).mark_rect().encode(
    x=alt.X(
        'shirt_number:O',  # Use ordinal for shirt numbers
        title='Shirt Number',
        axis=alt.Axis(labelAngle=0, orient='top', ticks=False)
    ),
    y=alt.Y(
        'game_label:N',
        sort=alt.EncodingSortField(field='date', order='descending'),
        axis=alt.Axis(labelExpr="slice(datum.value,15)", ticks=False, labelAngle=0, labelPadding=10, labelFontSize=11),
        title=None
    ),
    detail='game_id:N',
    color=alt.Color('player:N', scale=alt.Scale(scheme='category20c', domain=player_primary_position.sort_values('position', ascending=True)["player"].unique()), legend=None),
    opacity=alt.condition(click_player_highlight, alt.value(1.0), alt.value(0)),
    stroke=alt.condition(click_player_highlight, alt.value('black'), alt.value(None)),
    tooltip=[
        alt.Tooltip('player:N', title='Player'),
        alt.Tooltip('position_label:N', title='Position'),
        alt.Tooltip('game_type:N', title='Competition'),
        alt.Tooltip('date_str:N', title='Date'),
        alt.Tooltip('game_label_short:N', title='Opposition'),
    ]
).add_params(
    player_highlight, click_player_highlight
)

# Player name text
text = alt.Chart(df).mark_text(
    fontSize=12,
    baseline='middle',
    align='center',
    font='PT Sans Narrow'
).encode(
    x=alt.X('shirt_number:O', title=None),
    y=alt.Y('game_label:N', sort=alt.EncodingSortField(field='date', order='descending')),
    text=alt.Text('player_label:N'),
    color=alt.value('black'),
    opacity=alt.condition(click_player_highlight, alt.value(1.0), alt.value(0.5)),
    strokeWidth=alt.value(0.5),
    detail='game_id:N'
)

team_sheets = (chart + text).properties(
    width=alt.Step(45),
    height=alt.Step(20),
).facet(
    row=alt.Row('season:N', title=None, sort=alt.EncodingSortField(field='season', order='descending')),
    column=alt.Column('squad:N', title=None, header=alt.Header(title=None, labelFontSize=36, labelExpr="datum.value + ' XV'", labelAnchor="start")),
    spacing=10
).resolve_scale(y='independent', x='independent').properties(
    title=alt.Title("Team Sheets", subtitle="Player appearances by shirt number. Click on a player to highlight their appearances.")
).configure_view(strokeWidth=0)

team_sheets.save('python/team_sheets.json')

# Results

In [70]:
df = db.con.execute(
    """
    SELECT
        game_id,
        date,
        squad,
        opposition,
        home_away,
        CONCAT_WS(' ', opposition, CONCAT('(', home_away, ')')) AS game_label,
        season,
        pf,
        pa,
        result,
        competition,
        game_type,
        CASE
            WHEN pf >= pa THEN pf
            ELSE pa
        END AS winner,
        CASE
            WHEN pf < pa THEN pf
            ELSE pa
        END AS loser,
        ABS(pf - pa) AS margin
    FROM games
    """
).df()

team_filter = alt.selection_point(fields=["opposition"], on="hover", clear="mouseout", empty="all")
selection = alt.selection_point(fields=['result'], bind='legend')

base = alt.Chart(df).encode(
    y=alt.Y(
        "game_label:N", 
        title=None, 
        sort=alt.EncodingSortField(field='date', order='descending'), 
        axis=alt.Axis(
            title=None, 
            offset=15, 
            grid=False,
            ticks=False, 
            domain=False, 
        )
    ),
    color=alt.Color('result:N', scale=alt.Scale(domain=['W', 'L'], range=['#146f14', '#981515']), 
                legend=alt.Legend(orient="right", title="Result", titleOrient="top")),
    opacity=alt.condition(team_filter, alt.value(1.0), alt.value(0.2)),
)

bar = base.mark_bar().encode(
    x=alt.X('pf:Q', title="Points", axis=alt.Axis(orient='bottom', offset=5)),
    x2=alt.X2("pa:Q"),
).properties(
    width=400,
    height=alt.Step(15),
)
loser = base.mark_text(align='right', dx=-2, dy=0).encode(
    x=alt.X('loser:Q', title=None, axis=alt.Axis(orient='top', offset=5)),
    text='loser:N',
    color=alt.value('black')
)

winner = base.mark_text(align='left', dx=2, dy=0).encode(
    x=alt.X('winner:Q', title=None, axis=alt.Axis(orient='top', offset=5)),
    text='winner:N',
    color=alt.value('black')
)

(bar + loser + winner).transform_filter(selection).facet(
    row=alt.Row('season:N', title=None, sort=alt.EncodingSortField(field='season', order='descending')),
    column=alt.Column('squad:N', title=None, header=alt.Header(title=None, labelFontSize=36, labelExpr="datum.value + ' XV'", labelAnchor="start")),
    spacing=10
).resolve_scale(
    x='independent', y='independent'
).add_params(
    team_filter, selection
).properties(
    title=alt.Title("Results", subtitle="Points for and against in each game. Hover to highlight all games against a team. Click on legend to filter by result.")
).configure_view(strokeWidth=0).save('python/results.json')


# Set Piece Results

In [244]:
df = db.con.execute(
"""

SELECT
    S.*,
    G.squad,
    G.date,
    G.opposition,
    G.home_away,
    REPLACE(CONCAT_WS(' ', G.game_id, CONCAT('(', G.home_away, ')')), '_', ' ') AS game_label,
    CONCAT_WS(' ', G.opposition, CONCAT('(', G.home_away, ')')) AS game_label_short,
    G.season,
    G.game_type
FROM set_piece_stats S
LEFT JOIN games G USING (game_id)
ORDER BY G.date DESC, S.set_piece ASC
""").df()

s = "Lineout"

df_s = df[df['set_piece'] == s].melt(
    id_vars=['game_id', 'date', 'squad', 'opposition', 'game_label', 'game_label_short', 'season', 'game_type', 'team'], 
    value_vars=['won', 'lost'], 
    var_name='outcome', 
    value_name='count'
)
df_s["team_out"] = df_s.apply(
    lambda x: "EG" if (x["team"] == "EG" and x["outcome"].lower() == "won") or (x["team"] == "Opp" and x["outcome"].lower() == "lost") else "Opp",
    axis=1
)
df_s["Turnover"] = df_s.apply(lambda x: "Retained" if x['team_out']==x['team'] else "Turnover", axis=1)
df_s["game_id"] = df_s.apply(lambda x: f"{x['game_label_short']}_{x['game_id']}", axis=1)

df_s["team_total"] = df_s.groupby(['game_id', 'team'])['count'].transform('sum')
df_s["rate"] = df_s["count"] / df_s["team_total"]
df_s["team_name"] = df_s["team"].apply(lambda x: "East Grinstead" if x == "EG" else "Opposition")
df_s["team_out_name"] = df_s["team_out"].apply(lambda x: "East Grinstead" if x == "EG" else "Opposition")
df_s["success_rate"] = df_s.apply(lambda x: x["count"] / x["team_total"] if x["outcome"] == "won" else 1 - (x["count"] / x["team_total"]), axis=1)

# Pre-sort the dataframe to avoid complex sorting in Altair
df_s = df_s.sort_values(['date', 'squad'], ascending=[False, False]).reset_index(drop=True)
df_s["sort_order"] = df_s.index + 1
games_list = df_s['game_id'].unique().tolist()

turnover_filter = alt.selection_point(fields=["Turnover"], bind="legend")
put_in_filter = alt.selection_point(fields=["team"], bind="legend")
team_filter = alt.selection_point(fields=["opposition"])

#################
## Won vs Lost ##
#################
base = alt.Chart(df_s).encode(
    y=alt.Y(
        'game_id:N', 
        axis=None, 
        sort=games_list,
        scale=alt.Scale(padding=0),
        title=None
    ),
    opacity=alt.Opacity(
        "Turnover:N", 
        scale=alt.Scale(domain=["Turnover", "Retained"], range=[1.0, 0.5]),
        legend=alt.Legend(
            title="Result", 
            orient="right", 
            direction="vertical",
        ),
    ),
    tooltip=[
        alt.Tooltip('squad:N', title='Squad'),
        alt.Tooltip('season:N', title='Season'),
        alt.Tooltip('date:T', title='Date', format='%d %b %Y'),
        alt.Tooltip('game_label_short:N', title='Game'),
        alt.Tooltip('team_name:N', title='Attacking Team'),
        alt.Tooltip('team_out_name:N', title='Winning Team'),
        alt.Tooltip('count:Q', title='Count'),
    ],
    detail="opposition:N",
).properties(
    width=150,
    height=alt.Step(10),
).add_params(team_filter, turnover_filter, put_in_filter
).transform_filter(team_filter).transform_filter(turnover_filter).transform_filter(put_in_filter)

eg = base.mark_bar(stroke="#202946").encode(
    x=alt.X(
        'count:Q', 
        title=f'{s}s Won', 
        axis=alt.Axis(format='d', title="EG wins", titleColor="#202946", orient='top'), 
        scale=alt.Scale(domain=[0, df_s['count'].max()+1])
    ),
    yOffset=alt.YOffset('team:N', scale=alt.Scale(paddingOuter=0.2)),
    color=alt.Color(
        "team:N", 
        scale=alt.Scale(domain=['EG', 'Opp'], range=['#202946', '#981515']),
        legend=alt.Legend(title="Attacking team", orient="right", direction="horizontal")
    )
).transform_filter(alt.datum.team_out == 'EG')

opp = base.mark_bar(stroke="#981515").encode(
    x=alt.X(
        'count:Q', 
        title=f'{s}s Won', 
        axis=alt.Axis(format='d', title="Opposition wins", titleColor="#981515", orient='top'), 
        scale=alt.Scale(reverse=True, domain=[0, df_s['count'].max()+1]), 
        sort='descending'
    ),
    yOffset=alt.YOffset('team:N', scale=alt.Scale(paddingOuter=0.2)),
    color=alt.Color(
        "team:N", 
        scale=alt.Scale(domain=['EG', 'Opp'], range=['#202946', '#981515']),
        legend=alt.Legend(title="Attacking team", orient="right", direction="horizontal")
    ),
    y=alt.Y(
        'game_id:N', 
        axis=alt.Axis(orient="left", labelExpr="split(datum.value, '_')[0]", ticks=False, domain=False),
        sort=games_list,
        scale=alt.Scale(padding=0),
        title=None
    )
).transform_filter(alt.datum.team_out == 'Opp')

##################
## Success Rate ##
##################

df_success = df_s[df_s['outcome'] == 'won'].copy()
# Add a sort order field based on games_list position
game_sort_order = {game_id: i for i, game_id in enumerate(games_list)}
df_success['sort_order'] = df_success['game_id'].map(game_sort_order)
    
success_rate_eg = alt.Chart(df_success[df_success['team'] == 'EG']).mark_area(
    line={'color': '#202946'},
    point={'filled': True, 'color': '#202946'},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0), alt.GradientStop(color='#202946', offset=1)],
    )
).encode(
    x=alt.X("success_rate:Q", title="Success Rate", axis=alt.Axis(format='.0%', titleColor='black', orient='top'), scale=alt.Scale(domain=[0, 1])),
    y=alt.Y('game_id:N', axis=alt.Axis(orient="left", labels=False, ticks=False), sort=games_list, scale=alt.Scale(padding=0), title=None),
    yOffset=alt.YOffset('team:N', scale=alt.Scale(paddingOuter=0.2)),
    opacity=alt.value(0.75),
    tooltip=[
        alt.Tooltip('squad:N', title='Squad'),
        alt.Tooltip('season:N', title='Season'),
        alt.Tooltip('date:T', title='Date', format='%d %b %Y'),
        alt.Tooltip('game_label_short:N', title='Game'),
        alt.Tooltip('team_name:N', title='Attacking Team'),
        alt.Tooltip('team_total:Q', title=f'{s}s', format=',d'),
        alt.Tooltip('success_rate:Q', title='Success Rate', format='.0%')
    ],
)

success_rate_opp = alt.Chart(df_success[df_success['team'] == 'Opp']).mark_area(
    line={'color': '#981515'},
    point={'filled': True, 'color': '#981515'},
    color=alt.Gradient(
        gradient='linear',
        stops=[alt.GradientStop(color='white', offset=0), alt.GradientStop(color='#981515', offset=1)],
    )
).encode(
    x=alt.X("success_rate:Q", title="Success Rate", axis=alt.Axis(format='.0%', titleColor='black', orient='top'), scale=alt.Scale(domain=[0, 1])),
    y=alt.Y('game_id:N', axis=alt.Axis(orient="left", labels=False, ticks=False), sort=games_list, scale=alt.Scale(padding=0), title=None),
    yOffset=alt.YOffset('team:N', scale=alt.Scale(paddingOuter=0.2)),
    opacity=alt.value(0.75),
    tooltip=[
        alt.Tooltip('squad:N', title='Squad'),
        alt.Tooltip('season:N', title='Season'),
        alt.Tooltip('date:T', title='Date', format='%d %b %Y'),
        alt.Tooltip('game_label_short:N', title='Game'),
        alt.Tooltip('team_name:N', title='Attacking Team'),
        alt.Tooltip('team_total:Q', title=f'{s}s', format=',d'),
        alt.Tooltip('success_rate:Q', title='Success Rate', format='.0%')
    ],
)

success_rate_eg = success_rate_eg.transform_filter(team_filter).transform_filter(put_in_filter)
success_rate_opp = success_rate_opp.transform_filter(team_filter).transform_filter(put_in_filter)

###########
## FINAL ##
###########
(
    opp | eg | 
    (success_rate_eg + success_rate_opp).add_params(team_filter, put_in_filter)
).resolve_scale(
    x='independent', y='shared'
).properties(
    title=alt.Title(f"{s} Success", subtitle=[f"{s} results in terms of attacking team (EG in blue, opposition in red), and which team won the ball.", "Games are listed in chronological order with most recent at the top.", "Click the chart to filter by opposition. Click the legends to filter by attacking team or result."],)
).save(f'python/{s}_success.json')


# (success_rate_eg + success_rate_opp)

In [224]:
df_s

,game_id,date,squad,opposition,game_label,game_label_short,season,game_type,team,outcome,count,team_out,Turnover,team_total,rate,team_name,team_out_name,success_rate,sort_order
0,Beccehamians (H)_2025-08-23_1st_Beccehamians,2025-08-23,1st,Beccehamians,2025-08-23 1st Beccehamians (H),Beccehamians (H),2025/26,Friendly,EG,won,6,EG,Retained,9,0.666667,East Grinstead,East Grinstead,0.666667,1
1,Beccehamians (H)_2025-08-23_1st_Beccehamians,2025-08-23,1st,Beccehamians,2025-08-23 1st Beccehamians (H),Beccehamians (H),2025/26,Friendly,Opp,won,9,Opp,Retained,14,0.642857,Opposition,Opposition,0.642857,2
2,Beccehamians (H)_2025-08-23_1st_Beccehamians,2025-08-23,1st,Beccehamians,2025-08-23 1st Beccehamians (H),Beccehamians (H),2025/26,Friendly,Opp,lost,5,EG,Turnover,14,0.357143,Opposition,East Grinstead,0.642857,3
3,Beccehamians (H)_2025-08-23_1st_Beccehamians,2025-08-23,1st,Beccehamians,2025-08-23 1st Beccehamians (H),Beccehamians (H),2025/26,Friendly,EG,lost,3,Opp,Turnover,9,0.333333,East Grinstead,Opposition,0.666667,4
4,Haywards Heath (H)_2025-04-12_1st_Haywards_Heath,2025-04-12,1st,Haywards Heath,2025-04-12 1st Haywards Heath (H),Haywards Heath (H),2024/25,Cup,EG,lost,1,Opp,Turnover,11,0.090909,East Grinstead,Opposition,0.909091,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
371,Worthing (H)_2021-09-25_2nd_Worthing,2021-09-25,2nd,Worthing,2021-09-25 2nd Worthing (H),Worthing (H),2021/22,Friendly,EG,won,11,EG,Retained,14,0.785714,East Grinstead,East Grinstead,0.785714,372
372,Horsham II (A)_2021-09-04_1st_Horsham_II,2021-09-04,1st,Horsham II,2021-09-04 1st Horsham II (A),Horsham II (A),2021/22,Friendly,EG,won,15,EG,Retained,16,0.937500,East Grinstead,East Grinstead,0.937500,373
373,Horsham II (A)_2021-09-04_1st_Horsham_II,2021-09-04,1st,Horsham II,2021-09-04 1st Horsham II (A),Horsham II (A),2021/22,Friendly,Opp,won,11,Opp,Retained,13,0.846154,Opposition,Opposition,0.846154,374
374,Horsham II (A)_2021-09-04_1st_Horsham_II,2021-09-04,1st,Horsham II,2021-09-04 1st Horsham II (A),Horsham II (A),2021/22,Friendly,EG,lost,1,Opp,Turnover,16,0.062500,East Grinstead,Opposition,0.937500,375


In [191]:
df_success[(df_success['squad'] == '1st') & (df_success['team'] == 'EG')]

,game_id,date,squad,opposition,game_label,game_label_short,season,game_type,team,outcome,count,team_out,Turnover,team_total,rate,team_name,team_out_name,success_rate,game_order
0,Beccehamians (H)_2025-08-23_1st_Beccehamians,2025-08-23,1st,Beccehamians,2025-08-23 1st Beccehamians (H),Beccehamians (H),2025/26,Friendly,EG,won,6,EG,Retained,9,0.666667,East Grinstead,East Grinstead,0.666667,0
7,Haywards Heath (H)_2025-04-12_1st_Haywards_Heath,2025-04-12,1st,Haywards Heath,2025-04-12 1st Haywards Heath (H),Haywards Heath (H),2024/25,Cup,EG,won,10,EG,Retained,11,0.909091,East Grinstead,East Grinstead,0.909091,1
13,Cobham (H)_2025-03-22_1st_Cobham,2025-03-22,1st,Cobham,2025-03-22 1st Cobham (H),Cobham (H),2024/25,League,EG,won,15,EG,Retained,20,0.750000,East Grinstead,East Grinstead,0.750000,3
18,Old Rutlishians (A)_2025-03-08_1st_Old_Rutlish...,2025-03-08,1st,Old Rutlishians,2025-03-08 1st Old Rutlishians (A),Old Rutlishians (A),2024/25,League,EG,won,11,EG,Retained,14,0.785714,East Grinstead,East Grinstead,0.785714,4
21,Trinity (A)_2025-03-01_1st_Trinity,2025-03-01,1st,Trinity,2025-03-01 1st Trinity (A),Trinity (A),2024/25,League,EG,won,9,EG,Retained,11,0.818182,East Grinstead,East Grinstead,0.818182,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,Seaford (H)_2022-02-12_1st_Seaford,2022-02-12,1st,Seaford,2022-02-12 1st Seaford (H),Seaford (H),2021/22,League,EG,won,11,EG,Retained,12,0.916667,East Grinstead,East Grinstead,0.916667,84
355,Uckfield (H)_2021-11-13_1st_Uckfield,2021-11-13,1st,Uckfield,2021-11-13 1st Uckfield (H),Uckfield (H),2021/22,League,EG,won,9,EG,Retained,11,0.818182,East Grinstead,East Grinstead,0.818182,88
358,Burgess Hill (H)_2021-10-16_1st_Burgess_Hill,2021-10-16,1st,Burgess Hill,2021-10-16 1st Burgess Hill (H),Burgess Hill (H),2021/22,League,EG,won,5,EG,Retained,6,0.833333,East Grinstead,East Grinstead,0.833333,89
369,Eastbourne (H)_2021-09-25_1st_Eastbourne,2021-09-25,1st,Eastbourne,2021-09-25 1st Eastbourne (H),Eastbourne (H),2021/22,League,EG,won,15,EG,Retained,17,0.882353,East Grinstead,East Grinstead,0.882353,91


## --- IGNORE ---

In [ ]:
import pandas as pd
df = pd.read_csv('season_squad_analysis.csv')
df = df[(df["Team"] == "Hove") & (df["Season"] == "2024-2025")]
df["Start"] = df["Position"].apply(lambda x: True if x[0] != 'S' else False)
# Count starts and appearances
df["Appearances"] = 1
df = df.groupby("Player").agg({"Start": "sum", "Appearances": "count"}).sort_values("Start", ascending=False).reset_index()
df["Bench"] = df["Appearances"] - df["Start"]

highlight = [
    "Jordi Southgate",
    "Guy Patton",
    "Jack Edwards",
    "Leo Robertson",
    "Tom Langford",
    "Toby Gale",
    "Jordan Sayers",
    "Tobias Brown",
    "Taylor Morris",
    "Carl Buckland",
    "Alexander Milne",
    "Christopher Knott",
    "tyler ray hooper",
    "George Brown",
    "Issac Thomas",
    "Robin Ward",
    "Adam Whitbread",
    "Adrian Shergold",
    "Joe Hill",
]

df['Highlight'] = df['Player'].apply(lambda x: x in highlight)

# If player in highlight list not in the dataframe, add them with 0 starts and 0 appearances
for player in highlight:
    if player not in df['Player'].values:
        df = pd.concat([df, pd.DataFrame({"Player": [player], "Start": [0], "Appearances": [0], "Bench": [0], "Highlight": [True]})], ignore_index=True)

df

,Player,Start,Appearances,Bench,Highlight
0,Tom Holborn,20,20,0,False
1,Robin Ward,20,20,0,True
2,tyler ray hooper,19,19,0,True
3,Leo Robertson,19,19,0,True
4,Tobias Brown,16,16,0,True
5,Name withheld,16,16,0,False
6,John Orme-Williams,16,17,1,False
7,IAN SAUNDERS,16,16,0,False
8,George Brown,14,15,1,True
9,Ryan Storer,14,17,3,False


In [70]:
import altair as alt
from python.chart_helpers import *

# Stacked bar chart with Starts and Bench appearances per player
chart = alt.Chart(
    df.sort_values(['Highlight', 'Appearances'], ascending=[False, False])
).transform_fold(
    ['Start', 'Bench'],
    as_=['Appearance', 'Count'],
).mark_bar().encode(
    x=alt.X('sum(Count):Q', axis=alt.Axis(title=None, orient="top")),
    y=alt.Y(
        'Player:N',
        sort=alt.EncodingSortField(
            field='Highlight',
            order='descending',
            op='max'
        ),
        title=None
    ),
    color=alt.Color('Appearance:N',
        scale=alt.Scale(domain=['Start', 'Bench'], range=['#202946', '#7d96e8']),
        legend=alt.Legend(title=None, direction="horizontal", titleOrient="left", orient="bottom", labelExpr="datum.value == 'Start' ? 'Started' : 'Bench'"),
    ),
    opacity=alt.Opacity('Highlight:N', scale=alt.Scale(domain=[True, False], range=[1.0, 0.4]), legend=None),
    order=alt.Order('Appearance:N', sort='descending'),
    tooltip=[
        alt.Tooltip('Player:N', title='Player'),
        alt.Tooltip('sum(Count):Q', title='Appearances'),
        alt.Tooltip('Appearance:N', title='Type')
    ]
).properties(
    width=400,
    height=alt.Step(15),
    title='Player Appearances'
)

chart

alt.Chart(...)

In [188]:
tdf = pd.DataFrame(
    columns=["Week Commencing",	"Tuesday",	"Thursday"],
    data=[
        ["7 Jul", 0, 22],
        ["14 Jul", 16, 23],
        ["21 Jul", 21, 22],
        ["28 Jul", 24, 21],
        ["4 Aug", 23, 26],
        ["11 Aug", 24, 27],
        ["18 Aug", 27, 29],
        ["25 Aug", 24, 24],
        ["1 Sep", 14, 25],
        ["8 Sep", 21, 31]
    ]
)

tdf["Week Commencing"] = pd.to_datetime(tdf["Week Commencing"] + " 2025", format="%d %b %Y")
tdf = tdf.melt(id_vars=["Week Commencing"], value_vars=["Tuesday", "Thursday"], var_name="Day", value_name="Attendance")    
tdf["Date"]= tdf["Week Commencing"] + pd.to_timedelta(tdf["Day"].map({"Tuesday": 1, "Thursday": 3}), unit='D')
tdf["Week"] = tdf["Week Commencing"].dt.strftime('%d %b')
tdf

,Week Commencing,Day,Attendance,Date,Week
0,2025-07-07,Tuesday,0,2025-07-08,07 Jul
1,2025-07-14,Tuesday,16,2025-07-15,14 Jul
2,2025-07-21,Tuesday,21,2025-07-22,21 Jul
3,2025-07-28,Tuesday,24,2025-07-29,28 Jul
4,2025-08-04,Tuesday,23,2025-08-05,04 Aug
5,2025-08-11,Tuesday,24,2025-08-12,11 Aug
6,2025-08-18,Tuesday,27,2025-08-19,18 Aug
7,2025-08-25,Tuesday,24,2025-08-26,25 Aug
8,2025-09-01,Tuesday,14,2025-09-02,01 Sep
9,2025-09-08,Tuesday,21,2025-09-09,08 Sep


In [198]:
def chart(day, color, dx=0, dy=-8, add_text=True):

    chart = alt.Chart(tdf[tdf["Day"] == day]).encode(
        x=alt.X('Date:T', title="Training Week", axis=alt.Axis(format='%d %b', tickCount=5, grid=False)),
        y=alt.Y('Attendance:Q', title='Attendance', axis=alt.Axis(tickCount=4), scale=alt.Scale(domainMin=10, clamp=True, nice=True)).stack(None),
    ).transform_filter(
        alt.datum.Attendance > 0
    )

    area = chart.mark_area(
        line={'color': color},
        opacity=0.5,
        color=alt.Gradient(
            gradient='linear',
            stops=[alt.GradientStop(color='white', offset=0),
                   alt.GradientStop(color=color, offset=1)],
            x1=1,
            x2=1,
            y1=1,
            y2=0
        )
    )

    text = chart.mark_text(
        align='center',
        baseline='bottom',
        fontSize=12,
        dy=dy,
        dx=dx,
        color=color
    ).encode(
        text=alt.Text('Attendance:Q'),
    )

    return area + text if add_text else area

tues = chart("Tuesday", "darkgreen", add_text=False)

thurs = chart("Thursday", "darkblue", dx=0)

# Layer the two charts and add a color legend (Tuesday = dark green, Thursday = dark blue)
(tues + thurs).properties(
    width=500,
    height=300,
    title=alt.Title(
        "Training Attendance", subtitle="Senior players attending training on Tuesdays (green) and Thursdays (blue)", 
        subtitlePadding=5
    )  
).save('training_attendance.html')